### Bag of words

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [14]:
messages = ["Hey hey hey lets go get lunch today :)",
           "Did you go home?",
           "Hey!!! I need a favor"]
vect = CountVectorizer()
vect.fit(messages)
print(vect)
print("==========feature names=========")
print(vect.get_feature_names())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
==========feature names=========
['did', 'favor', 'get', 'go', 'hey', 'home', 'lets', 'lunch', 'need', 'today', 'you']


## 由上面的结果可知，主要作了以下处理:
- 单词全部转换为小写
- 小于2个字符的单词被过滤掉
- 删除了puncutation
- 单词去重

In [15]:
## fit 是用于拟合向量器的，当拟合过后，就可以用向量器来向量化文本。
dtm = vect.transform(messages)
print(dtm)
data = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
print("==========term frequency matrix=========")
print(data)

  (0, 2)	1
  (0, 3)	1
  (0, 4)	3
  (0, 6)	1
  (0, 7)	1
  (0, 9)	1
  (1, 0)	1
  (1, 3)	1
  (1, 5)	1
  (1, 10)	1
  (2, 1)	1
  (2, 4)	1
  (2, 8)	1
==========term frequency matrix=========
   did  favor  get  go  hey  home  lets  lunch  need  today  you
0    0      0    1   1    3     0     1      1     0      1    0
1    1      0    0   1    0     1     0      0     0      0    1
2    0      1    0   0    1     0     0      0     1      0    0


In [16]:
## 要注意的是，如果新的message里包含有己经拟合的vect中没有的单词，新的单词不会被加到结果
new_message=['Hey lets go get a drink tonight']
new_dtm = vect.transform(new_message)
print(new_dtm)
new_data = pd.DataFrame(new_dtm.toarray(), columns=vect.get_feature_names())
print(new_data)

  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 6)	1
   did  favor  get  go  hey  home  lets  lunch  need  today  you
0    0      0    1   1    1     0     1      0     0      0    0


In [17]:
### 全量拟合
messages.extend(new_message)
vect.fit(messages)
dtm = vect.transform(messages)
data = pd.DataFrame(dtm.toarray(), columns = vect.get_feature_names())
print(data)

   did  drink  favor  get  go  hey  home  lets  lunch  need  today  tonight  \
0    0      0      0    1   1    3     0     1      1     0      1        0   
1    1      0      0    0   1    0     1     0      0     0      0        0   
2    0      0      1    0   0    1     0     0      0     1      0        0   
3    0      1      0    1   1    1     0     1      0     0      0        1   

   you  
0    0  
1    1  
2    0  
3    0  


## 关于new_message里有新字段
这在做文档预测时，过滤掉测试集里在训练集里没有的字段，并不会有什么影响，因为即使加进去，训练出来的字段对应的权重w也是0。
伪代码:
    # creating DTMs
    vect = CountVectorizer()
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.fit(X_test)
    
    #creating and training logistic regression model
    logreg = LogisticRegression()
    logreg.fit(X_train_dtm, y_train)
    y_predicted = logreg.predict(X_test_dtm)  # predicting